In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgbm
from xgboost import plot_importance
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_supplement.csv', 'train.csv', 'test.csv', 'sample_submission.csv', 'train_sample.csv']


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
is_valid = False

path = '../input/'

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,131886954), nrows=53016937, usecols=train_columns, dtype=dtypes)
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)


CPU times: user 2min 9s, sys: 15.5 s, total: 2min 25s
Wall time: 2min 22s


In [ ]:
test.head()

In [3]:
Y=train.is_attributed
train.drop('is_attributed',axis=1,inplace=True)

In [4]:
def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["hour"]      = df["datetime"].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['datetime'], axis=1, inplace=True)
    return df


In [5]:
test=timeFeatures(test)
train=timeFeatures(train)

In [6]:
nrow_train = train.shape[0]
merge = pd.concat([train, test])
gc.collect()

247

In [8]:

###Experiment
# Count the number of clicks by ip
ip_count = merge.groupby(['ip','device','os'])['channel'].count().reset_index()
ip_count.columns = ['ip','device','os', 'clicks_by_ip_os_device']
merge = pd.merge(merge, ip_count, on=['ip','device','os'], how='left', sort=False)
merge['clicks_by_ip_os_device'] = merge['clicks_by_ip_os_device'].astype('uint16')
del ip_count
gc.collect()

42

In [11]:
###Experiment
ip_count = merge.groupby(['ip','device','os','app'])['channel'].count().reset_index()
ip_count.columns = ['ip','device','os','app', 'clicks_by_ip_os_device_app']
merge = pd.merge(merge, ip_count, on=['ip','device','os','app'], how='left', sort=False)
merge['clicks_by_ip_os_device_app'] = merge['clicks_by_ip_os_device_app'].astype('uint16')
del ip_count
gc.collect()

114

In [12]:
###Experiment
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
del ip_count
gc.collect()

35

In [ ]:
%%time
ip_count = merge.groupby(['ip','dow','hour'])['channel'].count().reset_index()
ip_count.columns = ['ip','dow','hour','clicks_by_ip_dow_hour']
merge = pd.merge(merge, ip_count, on=['ip','dow','hour'], how='left', sort=False)
merge['clicks_by_ip_dow_hour'] = merge['clicks_by_ip_dow_hour'].astype('uint16')
del ip_count
gc.collect()

In [ ]:
%%time

# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
del ip_count
gc.collect()

ip_count = merge.groupby(['ip','app'])['channel'].count().reset_index()
ip_count.columns = ['ip','app', 'clicks_by_ip_app']
merge = pd.merge(merge, ip_count, on=['ip','app'], how='left', sort=False)
merge['clicks_by_ip_app'] = merge['clicks_by_ip_app'].astype('uint16')
del ip_count
gc.collect()

ip_count = merge.groupby(['ip','os'])['channel'].count().reset_index()
ip_count.columns = ['ip','os', 'clicks_by_ip_os']
merge = pd.merge(merge, ip_count, on=['ip','os'], how='left', sort=False)
merge['clicks_by_ip_os'] = merge['clicks_by_ip_os'].astype('uint16')
del ip_count
#merge.drop('ip', axis=1, inplace=True)
gc.collect()


In [ ]:
%%time

# Count the number of os by ip
ip_count = merge.groupby(['channel'])['os'].count().reset_index()
ip_count.columns = ['channel', 'clicks_by_channel']
merge = pd.merge(merge, ip_count, on='channel', how='left', sort=False)
merge['clicks_by_channel'] = merge['clicks_by_channel'].astype('uint16')
#merge.drop('channel', axis=1, inplace=True)
del ip_count
print('channel done')

ip_count = merge.groupby(['os'])['channel'].count().reset_index()
ip_count.columns = ['os', 'clicks_by_os']
merge = pd.merge(merge, ip_count, on='os', how='left', sort=False)
merge['clicks_by_os'] = merge['clicks_by_os'].astype('uint16')
#merge.drop('os', axis=1, inplace=True)
del ip_count
print('os done')
gc.collect()

ip_count = merge.groupby(['app'])['channel'].count().reset_index()
ip_count.columns = ['app', 'clicks_by_app']
merge = pd.merge(merge, ip_count, on='app', how='left', sort=False)
merge['clicks_by_app'] = merge['clicks_by_app'].astype('uint16')
#merge.drop('app', axis=1, inplace=True)
del ip_count

print('app done')
#del ip_count,merge
#gc.collect()

In [13]:

train = merge[:nrow_train]
test = merge[nrow_train:]

del merge

gc.collect()

19

In [15]:
test.head()

,ip,app,device,os,channel,click_time,dow,hour,clicks_by_ip_os_device,clicks_by_ip_os_device_app,clicks_by_ip
53016937,5744,9,1,3,107,2017-11-10 04:00:00,4,4,6,3,469
53016938,119901,9,1,3,466,2017-11-10 04:00:00,4,4,164,14,7918
53016939,72287,21,1,19,128,2017-11-10 04:00:00,4,4,1030,52,5719
53016940,78477,15,1,13,111,2017-11-10 04:00:00,4,4,995,45,4513
53016941,123080,12,1,13,328,2017-11-10 04:00:00,4,4,158,27,583


In [16]:
train['click_time'] = pd.to_datetime(train['click_time'])
test['click_time'] = pd.to_datetime(test['click_time'])

In [17]:
def get_deltatime(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip','device','os']},
       ]

    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df.groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds
        
    df.drop(['click_time'],axis=1,inplace=True)
    return df

In [19]:
gc.collect()

0

In [18]:
test.reset_index(inplace=True,drop=True)


In [20]:

x_train=train[:47715243]
y_train=Y[:47715243]
x_valid=train[47715243:]
y_valid=Y[47715243:]

In [21]:
x_train.shape,x_valid.shape,y_train.shape,y_valid.shape

((47715243, 11), (5301694, 11), (47715243,), (5301694,))

In [ ]:
%%time
x_train=get_deltatime(x_train)

>> Grouping by ['ip', 'device', 'os'], and saving time to next click in: ip_device_os_nextClick


In [ ]:
%%time
x_valid=get_deltatime(x_valid)

In [ ]:
x_train.drop(['ip'],axis=1,inplace=True)
x_valid.drop(['ip'],axis=1,inplace=True)
gc.collect()

In [ ]:
    
dtrain = lgbm.Dataset(x_train, label=y_train)
dvalid = lgbm.Dataset(x_valid, label=y_valid)

In [ ]:
del x_train,x_valid,y_train,y_valid

In [ ]:
gc.collect()

In [ ]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves':15,  
    'max_depth': 4,
    #'scale_pos_weight': 99.7
    }


evals_results = {}

print("Training the model...")

lgb_model = lgbm.train(params, dtrain,valid_sets=[dtrain, dvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=200)

In [ ]:
print('Feature names:', lgb_model.feature_name())
print('Feature importances:', list(lgb_model.feature_importance()))


In [ ]:
submit = pd.read_csv(path_test, dtype='int', usecols=['click_id'])
submit['is_attributed'] = lgb_model.predict(test[predictors], num_iteration=lgb_model.best_iteration)
submit.to_csv('submission.csv', index=False)


In [ ]:

del train
del val
gc.collect()
